In [19]:
import tweepy

## A. Use a Twitter account* to register a developer app on https://apps.twitter.com, and select a hashtag to access data from for storage and analysis.

> We will use the #digitalhealth hashtag

## B. Perform historic data capture from the selected hashtag feed and store the data as a JSON file.

In [5]:
# Python Script to Extract tweets of a 
# particular Hashtag using Tweepy and Pandas
  
  
# import modules
import pandas as pd
import tweepy
  
  
# function to display data of each tweet
def printtweetdata(n, ith_tweet):
    pass
  
  
# function to perform data extraction
def collecte(words, date_since, numtweet):
      
    # Creating DataFrame using pandas
    db = pd.DataFrame(columns=['username', 'description', 'location', 'following',
                               'followers','timestamp', 'mentions', 'totaltweets', 'retweetcount', 'text', 'hashtags'])
      
    # We are using .Cursor() to search through twitter for the required tweets.
    # The number of tweets can be restricted using .items(number of tweets)
    tweets = tweepy.Cursor(api.search, q=words, lang="en",
                           since=date_since, tweet_mode='extended').items(numtweet)
     
    # .Cursor() returns an iterable object. Each item in 
    # the iterator has various attributes that you can access to 
    # get information about each tweet
    list_tweets = [tweet for tweet in tweets]
      
    # Counter to maintain Tweet Count
    i = 1  
      
    # we will iterate over each tweet in the list for extracting information about each tweet
    for tweet in list_tweets:
        username = tweet.user.screen_name
        description = tweet.user.description
        location = tweet.user.location
        following = tweet.user.friends_count
        followers = tweet.user.followers_count
        timestamp = tweet.created_at
        user_mentions = tweet.entities['user_mentions']
        if user_mentions:
          mentions = [user['screen_name'] for user in user_mentions] # name, id, screen_name
        else:
          mentions = user_mentions
        totaltweets = tweet.user.statuses_count
        retweetcount = tweet.retweet_count
        hashtags = tweet.entities['hashtags']

          
        # Retweets can be distinguished by a retweeted_status attribute,
        # in case it is an invalid reference, except block will be executed
        try:
            text = tweet.retweeted_status.full_text
        except AttributeError:
            text = tweet.full_text
        hashtext = list()
        for j in range(0, len(hashtags)):
            hashtext.append(hashtags[j]['text'])
          
        # Here we are appending all the extracted information in the DataFrame
        ith_tweet = [username, description, location, following,
                     followers,timestamp,mentions, totaltweets, retweetcount, text, hashtext]
        db.loc[len(db)] = ith_tweet
          
        # Function call to print tweet data on screen
        printtweetdata(i, ith_tweet)
        i = i+1
    filename = 'collected_tweets.csv'
      
    # we will save our database as a CSV file.
    db.to_json('historical_tweets.json', orient='columns')
  
  
if __name__ == '__main__':
      
    # Enter your own credentials obtained 
    # from your developer account
    consumer_key = "Plf6N9ZtnkmnXeHa5qIgOJipt"
    consumer_secret = "W0IcerPxfVeSjMUlNTEL5FaDJ9HVHxikQV60N4ee1fnljjasI6"
    access_key = "818345262221590528-eduHlh4TPISFyueZWonEUgnHNcVVx1Y"
    access_secret = "XwwEApZw8OXIK5JJLoya6M5FkICoiR45UEqHj0D9phO9L"
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
      
    # Enter Hashtag and initial date
    print("Enter Twitter HashTag to search for (exclude the # symbole for instance use digitalhealth instead of #digitalhealth")
    words = input()
    print("Enter Date since The Tweets are required in yyyy-mm--dd")
    date_since = input()
      
    # number of tweets you want to extract in one run
    print('Number of tweets to scrape')
    numtweet = int(input())
    collecte(words, date_since, numtweet)
    print('collecting has completed!')

Enter Twitter HashTag to search for (exclude the # symbole for instance use digitalhealth instead of #digitalhealth
design
Enter Date since The Tweets are required in yyyy-mm--dd
2020-01-01
Number of tweets to scrape
5000


Rate limit reached. Sleeping for: 710


collecting has completed!


In [4]:
import pandas as pd
#Read the collected json file
df = pd.read_json('historical_tweets.json')

In [116]:
df.head()

,username,description,location,following,followers,timestamp,mentions,totaltweets,retweetcount,text,hashtags
0,IJimin_everyday,💜BTS Army 🪐 OT7 💫 I Jim-in’d & I'll never Jim-...,Joonie’s precious dimples,1042,319,2021-10-18 01:19:47,"[Sonyeondamn7, sugatalus, naija0329]",11310,11,@sugatalus @naija0329 If a swiss knife was a p...,[]
1,tv_imasu,main: @usami_vt * alt account! slightly irl an...,,66,268,2021-10-18 01:19:46,[],5726,0,i wanna get more art of the design,[]
2,RoseTintedMoon,Rose | 21 | He/They/Her | 💖 @SereneTrash 💖,"Arizona, USA",961,381,2021-10-18 01:19:45,[minbitt],47395,3190,swatch's design is just so 🥺💙 https://t.co/lDZ...,[]
3,uncompletd,Photographer/Creative,"Detroit, MI",846,2641,2021-10-18 01:19:43,[],83102,0,"I really love simplicity. Outfits, design, art...",[]
4,weems,he/him. crypto means encryption and is not a c...,"(35.7004917, 139.7717283)",4998,2137,2021-10-18 01:19:43,[Chinchillazllla],224451,5696,"in 2008, Vermont prison inmates were being use...",[]


In [9]:
df.columns

Index(['username', 'description', 'location', 'following', 'followers',
       'timestamp', 'mentions', 'totaltweets', 'retweetcount', 'text',
       'hashtags'],
      dtype='object')

In [5]:
import re
#1-Cleaning text column to remove user mentions from text column

clean_text = [re.sub('@[A-Za-z0-9]+', '', item) for item in df.text]

#2- Remove links from tweet text

clean_text = [re.sub(r"http\S+", "", subject) for subject in clean_text]

df['clean_text'] = clean_text
#3- Remove punctuations
df["clean_text"] = df['clean_text'].str.replace('[^\w\s]','')

import string
#Remove the square brackets by converting row lists to string
df['mentions'] = [','.join(item) for item in df['mentions']]


## 4- Remove stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
#Remove stop words
df['clean_text']= df['clean_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))


#Extract hour and month from the data
#month
df['month'] = df.timestamp.dt.month
#hour
df['hour'] = df.timestamp.dt.hour

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Frequency of mentions of organisation(s) of your choice.

In [6]:
dfx = df[['mentions']]
#Set splitting value to 50 to make sue we capture all the mentions
dfx = dfx['mentions'].str.split(",", n = 50, expand = True)

orgs = dfx.to_numpy().flatten().tolist()

#remove Nones
orgs = [x for x in orgs if x is not None]
dff = pd.DataFrame()
dff['orgs'] = orgs
#Group by to obtain count of each organization/person
orgs1 = pd.DataFrame(dff.groupby('orgs')['orgs'].count())
orgs1.columns = ['Count']
orgs1 = orgs1.reset_index()
#Define length column to help remove organizations with no names later
orgs1['len'] =  [len(item) for item in orgs1['orgs']]
orgs1 = orgs1[orgs1['len'] > 0]
#Drop the length column
orgs1 = orgs1.drop('len', axis = 1)
selectt = orgs1.sort_values('Count', ascending = False)
selectt1 = selectt[(selectt['orgs'] == 'NECA_TOYS') | (selectt['orgs'] == 'today_crypto')]

In [135]:
selectt1

,orgs,Count
2446,today_crypto,139
895,NECA_TOYS,21


### Frequency of selected meaningful text term(s).

In [18]:
#Split to obtain all the terms
term_df = df.clean_text.str.split(expand=True).stack().value_counts().reset_index()
#Get frequency of all terms
term_df.columns = ['Word', 'Frequency']
term_df['Word'] = [item.title() for item in term_df['Word']]
#Define meaningful test terms
terms = ['Machinelearning', 'Ai', 'Bigdata','Nfts',"Design", "Iot", 'Datamining', 'Covid', 'Covid19', 'Coronavirus', 'Corona']
 
key_terms = term_df[term_df['Word'].isin(terms)].reset_index()
key_terms = key_terms.drop('index', axis = 1)
key_terms = pd.DataFrame(key_terms.groupby('Word')['Frequency'].sum()).reset_index()
key_terms.sort_values('Frequency', ascending = False)

,Word,Frequency
4,Design,5092
7,Nfts,327
6,Machinelearning,19
0,Ai,11
2,Covid,7
3,Covid19,3
1,Coronavirus,2
5,Iot,1


In [11]:
term_df.sort_values('Word', ascending = True)

,Word,Frequency
707,0,16
8173,0001,1
13900,000326,1
14692,0005,1
12419,0012,1
...,...,...
6952,𝙾𝙲𝚃,2
12272,𝟚𝟛𝕣𝕕,1
7481,𝟚𝟜𝕥𝕙,1
11218,𝟶𝟿𝙿𝙼,1


### Most used @ mentions

In [138]:
dfx = df[['mentions']]
#Set splitting value to 50 to make sue we capture all the mentions
dfx = dfx['mentions'].str.split(",", n = 50, expand = True)

orgs = dfx.to_numpy().flatten().tolist()

#remove Nones
orgs = [x for x in orgs if x is not None]
dff = pd.DataFrame()
dff['orgs'] = orgs
#Group by to obtain count of each organization/person
orgs1 = pd.DataFrame(dff.groupby('orgs')['orgs'].count())
orgs1.columns = ['Count']
orgs1 = orgs1.reset_index()
#Define length column to help remove entities with no names later
orgs1['len'] =  [len(item) for item in orgs1['orgs']]
#Subset to remove entities that do not have names
orgs1 = orgs1[orgs1['len'] > 0]
#Drop the length column
orgs1 = orgs1.drop('len', axis = 1)
selectt = orgs1.sort_values('Count', ascending = False)
#Top 20 most used @ mentions
selectt.head(20)

,orgs,Count
261,Chinchillazllla,366
2446,today_crypto,139
1494,actorvijay,106
1609,bugjamborees,99
1400,WillowCreative_,61
2105,minbitt,57
1932,its_Kamal_vj,41
1253,Team_CDT,38
1980,just_oyinda,38
1217,StrawderryST,37


## C. Perform analysis of streaming data from the selected hashtag feed (minimum of 10,000 real-time tweets), explaining the method and application

### Collect data

In [94]:
%%time
import tweepy
import csv
from tweepy.streaming import StreamListener

#pass security information to variables
consumer_key = "Plf6N9ZtnkmnXeHa5qIgOJipt"
consumer_secret = "W0IcerPxfVeSjMUlNTEL5FaDJ9HVHxikQV60N4ee1fnljjasI6"
access_key = "818345262221590528-eduHlh4TPISFyueZWonEUgnHNcVVx1Y"
access_secret = "XwwEApZw8OXIK5JJLoya6M5FkICoiR45UEqHj0D9phO9L"


#use variables to access twitter
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

#create an object called 'TweetStreamer'

class TweetStreamer(StreamListener):
    def __init__(self):
      super().__init__()
      self.counter = 0
      #Define the number of tweets to stream (in this case 10000 tweets)
      self.limit = 10000

    def on_status(self, status):
        # Writing status data
        with open('OutputStreaming1.csv', 'a') as f:
            writer = csv.writer(f)
            hashtags = status.entities['hashtags']
            hashtext = list()
            hashh = []
            for j in range(0, len(hashtags)):
                hashtext.append(hashtags[j]['text'])
            hashh.append(hashtext)
            writer.writerow([status.author.screen_name, status.created_at,status.user.followers_count,hashtext, status.text])
            self.counter += 1
            if self.counter < self.limit:
                return True
            else:
                streamingAPI.disconnect()
            


    def on_error(self, status_code):
        print >> sys.stderr, 'Encountered error with status code:', status_code
        return True # Don't kill the stream

    def on_timeout(self):
        print >> sys.stderr, 'Timeout...'
        return True # Don't kill the stream

runtime = 60 #this means one minute
streamingAPI = tweepy.streaming.Stream(auth, TweetStreamer())
print('Input the hashtag you would like to track such as design, healthcare, etc')
hastagg = input()
streamingAPI.filter(track=[hastagg])

Input the hashtag you would like to track such as design, healthcare, etc
design
CPU times: user 92.3 ms, sys: 9.58 ms, total: 102 ms
Wall time: 15.1 s


In [2]:
import pandas as pd
df = pd.read_csv('OutputStreaming.csv', header= None)
df.columns = ['user', 'created_at', 'followers_count','hashtag', 'text']
df.head(10)

,user,created_at,followers_count,hashtag,text
0,pngurevor,2021-10-18 02:03:38,52,[],Kung kelan gusto ko mag design ng mga necklace...
1,chestnut_baron,2021-10-18 02:03:38,23,[],@Magical__Hacker Seems pretty in line with the...
2,Board_4PF,2021-10-18 02:03:38,2335,[],RT @just_oyinda: Worked on this design all nig...
3,VillaCareKenya,2021-10-18 02:03:38,5652,[],SIMCO PLAZA-LUSAKA ROAD:The elegant and contem...
4,pkake2000,2021-10-18 02:03:39,1185,[],Best Auntie Ever Photo Frame 20oz Skinny Taper...
5,735_bruj,2021-10-18 02:03:40,1,[],"RT @AmorAltra: And here is mine, yup https://t..."
6,Kazudotid,2021-10-18 02:03:41,733,[],"terima jasa design logo, poster, dll by reques..."
7,TheOsseophile,2021-10-18 02:03:41,1967,[],RT @ExoSuitOne: Why can't we go back to styliz...
8,dartgirl42,2021-10-18 02:03:41,113,[],RT @joshhumble: It would be great if @Google a...
9,KTTHWLMJ2,2021-10-18 02:03:41,535,[],RT @KingdomArt5: Design concepts (Kingdom Hear...


###  A list of hashtags most used with your selected hashtag

In [3]:
import warnings
warnings.filterwarnings('ignore')
dfx = df[['hashtag']]
#Remove the square brackets by converting row lists to string
import string
def remove_punctuation(s):
    s = ''.join([i for i in s if i not in frozenset(string.punctuation)])
    return s

dfx['hashtag'] = dfx['hashtag'].apply(remove_punctuation)
#Set splitting value to 50 to make sure we capture all the hashtags
dfx = dfx['hashtag'].str.split(" ", n = 50, expand = True)

hash = dfx.to_numpy().flatten().tolist()

#remove Nones
hash = [x for x in hash if x is not None]
dff = pd.DataFrame()
dff['Hastags'] = hash
#Group by to obtain count of each organization/person
hash1 = pd.DataFrame(dff.groupby('Hastags')['Hastags'].count())
hash1.columns = ['Count']
hash1 = hash1.reset_index()
#Define length column to help remove entities with no names later
hash1['len'] =  [len(item) for item in hash1['Hastags']]
#Subset to remove entities that do not have names
hash1 = hash1[hash1['len'] > 0]
#Drop the length column
hash1 = hash1.drop('len', axis = 1)
selectt = hash1.sort_values('Count', ascending = False)
#Top 20 most used @ mentions
selectt.head(20)

,Hastags,Count
34,4YrsOfBlockbusterMersal,425
440,Master,417
113,Beast,339
448,Mersal,233
986,design,184
599,SHIB,147
624,Shiba,143
982,deltarune,135
1564,spamton,121
1811,サイストア,100


### Most occurring terms

In [4]:
import re
#1-Cleaning text column to remove user mentions from text column
clean_text = [re.sub('@[A-Za-z0-9]+', '', item) for item in df.text]

#2- Remove links from tweet text

clean_text = [re.sub(r"http\S+", "", subject) for subject in clean_text]

df['clean_text'] = clean_text
#3- Remove punctuations
df["clean_text"] = df['clean_text'].str.replace('[^\w\s]','')

## 4- Remove stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

df['clean_text']= df['clean_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
#Split to obtain all the terms
term_df = df.clean_text.str.split(expand=True).stack().value_counts().reset_index()
#Get frequency of all terms
term_df.columns = ['Word', 'Frequency']
#Get length of words and assume that important words have more than 3 letters
term_df['len'] = [len(item) for item in term_df['Word']]
term_df = term_df[term_df['len'] > 3]
#Drop the length variable
term_df = term_df.drop('len', axis = 1)
term_df.head(20)

,Word,Frequency
1,design,5630
3,Design,1583
5,logo,548
6,make,533
7,Check,530
9,like,459
10,Fiverr,452
11,Master,436
13,4YrsOfBlockbusterMersal,425
14,used,386


### Terms that may frequently occur together in the feed (bigram terms)

In [6]:
nltk.download('words')
words = set(nltk.corpus.words.words())
#Remove non-english words
df['cleantext'] = [" ".join(w for w in nltk.wordpunct_tokenize(item) if w.lower() in words or not w.isalpha()) for item in df['clean_text']]

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [9]:
from collections import Counter
from itertools import chain
#Define a function to generate the n-grams
def find_ngrams(input_list, n):
    return list(zip(*[input_list[i:] for i in range(n)]))

#Generate the bigrams by setting n to 2
df['bigrams'] = df['cleantext'].map(lambda x: find_ngrams(x.split(" "), 2))
# Bigram Frequency Counts
bigrams = df['bigrams'].tolist()
bigrams = list(chain(*bigrams))
bigrams = [(x.lower(), y.lower()) for x,y in bigrams]

bigram_counts = Counter(bigrams)
bigram_counts.most_common(20)

[(('check', 'gig'), 448),
 (('they', 'design'), 353),
 (('2008', 'prison'), 350),
 (('prison', 'used'), 350),
 (('used', 'labor'), 350),
 (('labor', 'make'), 350),
 (('make', 'cop'), 350),
 (('cop', 'they'), 350),
 (('design', 'hide'), 350),
 (('gig', 'create'), 349),
 (('professional', 'design'), 347),
 (('create', 'professional'), 345),
 (('design', 'copyright'), 345),
 (('copyright', '12'), 345),
 (('12', 'hour'), 345),
 (('character', 'design'), 333),
 (('beast', 'master'), 234),
 (('new', 'design'), 231),
 (('special', 'design'), 211),
 (('peak', 'character'), 160)]

In [12]:
#add to dataframe
usage = pd.DataFrame(bigram_counts.most_common(20))
usage.columns = ['bigram', 'Number of times used']
usage

,bigram,Number of times used
0,"(check, gig)",448
1,"(they, design)",353
2,"(2008, prison)",350
3,"(prison, used)",350
4,"(used, labor)",350
5,"(labor, make)",350
6,"(make, cop)",350
7,"(cop, they)",350
8,"(design, hide)",350
9,"(gig, create)",349
